In [4]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import requests
from io import BytesIO

# Загрузка предобученной модели ResNet
model = models.resnet18(pretrained=True)
model.eval()

# Загрузка и предобработка изображения
def process_image(image_path):
    input_image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

    return input_batch

# Функция для классификации изображения
def classify_image(image_path):
    input_batch = process_image(image_path)

    with torch.no_grad():
        output = model(input_batch)
    
    # Трансформируем выходные данные в вероятности
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    
    # Загрузка меток классов
    LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
    labels = requests.get(LABELS_URL).json()

    # Получение метки класса с максимальной вероятностью
    max_prob, max_idx = torch.max(probabilities, dim=0)
    predicted_label = labels[max_idx]

    return predicted_label, max_prob.item()

# Пример использования -- по папке пройтись
image_url = "https://cf.bstatic.com/xdata/images/hotel/max1024x768/143407681.jpg?k=1b31194500187b168af60058a36fe5482defae5fcd105a2b13356a83b694d28a&o=&hp=1"  # Замените на URL вашего изображения
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Сохранение изображения локально для последующей обработки
image_path = "image.jpg"
image.save(image_path)

# Классификация изображения
predicted_label, probability = classify_image(image_path)
print(f"Predicted label: {predicted_label}, Probability: {probability:.4f}")

Predicted label: palace, Probability: 0.4025
